## Leveraging AI for Knowledge Management at Insurellm
We implemented OpenAI's ChatGPT to structure and retrieve knowledge from Insurellm's shared folders, which included Company, Contracts, Employees, and Products. Initially, we collected subfolder and file data. Then, we chunked the documents to avoid memory issues and converted the text to vectors with FAISS. Next, we visualized the vector data with Plotly and utilized LangChain RAG for advanced retrieval. Finally, we deployed a multilingual chatbot via Gradio to enhance user interaction. This approach ensured data privacy while significantly improving knowledge management and operational efficiency.


In [ ]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr
import pandas as pd

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
# from langchain_chroma import Chroma
from langchain.vectorstores import FAISS
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [ ]:
MODEL = "gpt-4o-mini"
db_name = "vector_db"
load_dotenv()
os.environ['OPENAI_API_KEY'] = 'you api key'

In [ ]:
# The following code will be used to get all the subfolders' names and the file counts related to the folders
# Path to the main directory containing subfolders
main_directory = "knowledge-base"

# List to hold subfolder names and their file counts
data = []

# Loop through each subfolder in the main directory
for subfolder in os.listdir(main_directory):
    subfolder_path = os.path.join(main_directory, subfolder)
    
    if os.path.isdir(subfolder_path):
        # Count the number of files in the subfolder
        file_count = sum([len(files) for _, _, files in os.walk(subfolder_path)])
        data.append({"Subfolder": subfolder, "File Count": file_count})

# Create a DataFrame from the data
df = pd.DataFrame(data)

# Display the subfolder name and the file counts
print(df)


   Subfolder  File Count
0   products           4
1  contracts          13
2    company           3
3  employees          12


In [ ]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

folders = glob.glob("knowledge-base/*")

text_loader_kwargs = {'encoding': 'utf-8'}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

Large documents can exceed memory limits or the processing capacity of the system. By splitting them into smaller chunks, you ensure that the system can handle the text without running out of memory or crashing.

In [23]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1088, which is longer than the specified 1000


In [25]:
len(chunks)

123

In [26]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: company, contracts, products, employees


The following code takes a collection of text chunks, converts them into vectors using OpenAI embeddings, stores them in a FAISS vector store, and then retrieves and prints out the total number of vectors and their dimensions. This open-source vector encoder ensures that the data remains on our computer, which is crucial when building enterprise systems that require data to stay internal.

In [ ]:
import faiss
vectorstore = FAISS.from_documents(chunks, embedding=embeddings)


embeddings = OpenAIEmbeddings()


vectorstore = FAISS.from_documents(chunks, embedding=embeddings)

total_vectors = vectorstore.index.ntotal
dimensions = vectorstore.index.d

print(f"There are {total_vectors} vectors with {dimensions:,} dimensions in the vector store")

There are 123 vectors with 1,536 dimensions in the vector store


In [29]:
# Prework
vectors = []
documents = []
doc_types = []
colors = []
color_map = {'products':'blue', 'employees':'green', 'contracts':'red', 'company':'orange'}

for i in range(total_vectors):
    vectors.append(vectorstore.index.reconstruct(i))
    doc_id = vectorstore.index_to_docstore_id[i]
    document = vectorstore.docstore.search(doc_id)
    documents.append(document.page_content)
    doc_type = document.metadata['doc_type']
    doc_types.append(doc_type)
    colors.append(color_map[doc_type])
    
vectors = np.array(vectors)

The following part using Plotly to visualize the embeding vectors numbers and their related subfolders.

In [38]:
# Plotly 2D visualization.
tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D FAISS Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [39]:
# Plotly 3D visualization

tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D FAISS Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

## Time to use LangChain to bring it all together

In [ ]:
# create a new Chat using OpenAI LLM, temperatue is a parameter controls the randomness of the responses (0.7 is moderately creative)
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

/var/folders/6d/k1kp92h91dg01vysfjhpw89m0000gn/T/ipykernel_28630/4268784041.py:5: LangChainDeprecationWarning:

Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/



In [42]:
query = "Can you please summarize Insurellm in a few sentences?"
result = conversation_chain.invoke({"question":query})
result["answer"]

'Insurellm is an innovative insurance tech startup founded by Avery Lancaster in 2015, with a mission to disrupt the insurance industry through technology. The company offers four software products: Carllm for auto insurance, Homellm for home insurance, Rellm for the reinsurance sector, and Marketllm, a marketplace connecting consumers with insurance providers. With 200 employees and over 300 clients worldwide, Insurellm is dedicated to transforming the insurance landscape with reliability and innovation.'

In [ ]:
# The following code takes a collection of text chunks, converts them into vectors using OpenAI embeddings, 
# stores them in a FAISS vector store, and then retrieves and prints out the total number of vectors and 
# their dimensions. This open-source vector encoder ensures that the data remains on our computer, 
# which is crucial when building enterprise systems that require data to stay internal.

# create a new Chat with OpenAI
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)


In [45]:
# The following code takes a collection of text chunks, converts them into vectors using OpenAI embeddings, 
# stores them in a FAISS vector store, and then retrieves and prints out the total number of vectors and 
# their dimensions. This open-source vector encoder ensures that the data remains on our computer, 
# which is crucial when building enterprise systems that require data to stay internal.

# create a new Chat with OpenAI
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

# export result in markdown format
result = conversation_chain({"question": "can you describe Insurellm"})
result_markdown = f"```markdown\n{result['answer']}\n```"
print(result_markdown)


```markdown
Insurellm is an innovative insurance tech firm founded by Avery Lancaster in 2015. The company aims to disrupt the insurance industry with its innovative products. As of 2024, Insurellm has grown to employ 200 people and operates 12 offices across the United States. 

Insurellm offers four main software products:
1. Carllm - a portal for auto insurance companies.
2. Homellm - a portal for home insurance companies.
3. Rellm - an enterprise platform for the reinsurance sector.
4. Marketllm - a marketplace that connects consumers with insurance providers.

The company has more than 300 clients worldwide and focuses on transforming the insurance landscape with a commitment to innovation and reliability.
```


In [47]:
result = conversation_chain({"question": "can you describe Insurellm in Spanish"})
result_markdown = f"```markdown\n{result['answer']}\n```"
print(result_markdown)

```markdown
Insurellm es una empresa innovadora de tecnología de seguros fundada por Avery Lancaster en 2015. Su objetivo es transformar la industria de seguros con productos innovadores. La primera oferta de Insurellm fue Markellm, un mercado que conecta a los consumidores con proveedores de seguros. Desde entonces, la empresa ha crecido rápidamente, alcanzando 200 empleados para 2024 y estableciendo 12 oficinas en los Estados Unidos.

Insurellm ofrece cuatro productos de software de seguros:

- **Carllm**: un portal para compañías de seguros de automóviles.
- **Homellm**: un portal para compañías de seguros de hogar.
- **Rellm**: una plataforma empresarial para el sector de reaseguro.
- **Marketllm**: un mercado para conectar a los consumidores con proveedores de seguros.

La empresa cuenta con más de 300 clientes en todo el mundo y está comprometida con la innovación y la fiabilidad en el ámbito del seguro de hogar a través de Homellm.
```


In [46]:
result = conversation_chain({"question": "Who is CEO"})
result_markdown = f"```markdown\n{result['answer']}\n```"
print(result_markdown)

```markdown
The CEO of Insurellm is Avery Lancaster.
```


In [49]:
result = conversation_chain({"question": "Can you describe Avery Lancaster background information in a few sentences in Chinese?"})
result_markdown = f"```markdown\n{result['answer']}\n```"
print(result_markdown)

```markdown
Avery Lancaster是一位1985年3月15日出生的女性，目前担任Insurellm的联合创始人兼首席执行官（CEO）。她位于加利福尼亚州的旧金山。Avery于2015年共同创立了Insurellm，并在此期间引领公司成为领先的保险科技提供商。她以创新的领导策略和风险管理专长而闻名，将公司推向主流保险市场。

在创办Insurellm之前，Avery于2013年至2015年在Innovate Insurance Solutions担任高级产品经理，专注于开发面向科技行业的突破性保险产品。Avery积极参与领导力培训项目和行业会议，代表Insurellm并促进合作伙伴关系。她还倡导多样性和包容性，致力于改善团队的代表性，并通过实施灵活的工作条件来应对员工的工作与生活平衡问题。此外，Avery还领导社区参与活动，专注于面向服务不足人群的金融素养项目，提升了Insurellm的企业社会责任形象。
```


### Using Gradio to build a use friendly web interface for chatbot

In [34]:
# Wrapping that in a function

def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [35]:
# And in Gradio:

view = gr.ChatInterface(chat).launch()

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/gradio/components/chatbot.py:243: UserWarning:

The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.



* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
